In [13]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [14]:
from pathlib import Path
from src.utils.config_loader import load_config

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

In [15]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import (SystemMessage, HumanMessage)

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name=config['OPENAI_DEPLOYMENT_NAME'],
    temperature=0.0
)

In [16]:
import json

with open(base_dir / 'data/val_set.json', 'r') as f:
    data = json.load(f)

In [17]:
def format_examples(example_subset): 
    # Formats the examples into a string for later prompt
    formatted = []
    for i, ex in enumerate(example_subset):
        formatted.append(
            f"Eksempel {i+1}:\n"
            f"Versjon 1:\n{ex['version_1']}\n\n"
            f"Versjon 2:\n{ex['version_2']}\n\n"
            f"Oppsummering:\n{ex['ref_summary']}\n##\n"
        )
    
    return "\n".join(formatted)

#ids = [27]
ids = [14, 22, 15]

examples = [next(ex for ex in data if ex["id"] == id) for id in ids]

formatted_examples = format_examples(examples)

print(formatted_examples)

Eksempel 1:
Versjon 1:
5. Bestemmelser til bestemmelsesområder  5.1 Bestemmelser til bestemmelsesområde (område #1, #Masseutak) • Innenfor området tillates det uttak av masser.  • Anleggsarbeid skal ikke skje i påska (Skjærtorsdag - 2.påskedag), pinse (pinseaften 2.pinsedag), jul (Julaften - 2. juledag), nyttårshelg eller offentlige høytidsdager. • Driftstid skal avgrenses til 08.00-20.00, mandag - torsdag. • Ved støvplager skal det iverksetters avbøtende tiltak som for eksempel vanning, salting eller liknede • Uttaksområdet skal deles opp i etapper. Hver etappe skal ha en avslutningsplan. Ved opphør av utbygging over 2 år skal massetaket avsluttes og tiltakene i avslutningsplanen skal gjennomføres. 

Versjon 2:


Oppsummering:
Hele bestemmelsen for bestemmelsesområde #1 Masseutak er fjernet, inkludert tillatelse til masseuttak, arbeidstidsbegrensninger, krav om avbøtende tiltak, etappevis uttak og avslutningsplan.
##

Eksempel 2:
Versjon 1:
Belysning av fellesområder skal utformes sli

In [18]:
from tqdm import tqdm

with open(base_dir / 'data/val_set.json', 'r') as f:
    test_set = json.load(f)

results = []

data_sample = test_set

for item in tqdm(data_sample):

    version_1 = item['version_1']
    version_2 = item['version_2']
    ref_summary = item['ref_summary']

    msg = [
    SystemMessage(
        content=(
            "Du er en fagperson med ekspertise innen arealplanlegging og "
            "reguleringsplaner. Din oppgave er å bistå saksbehandlere med å "
            "identifisere og oppsummere forskjellene mellom to versjoner av "
            "tekst hentet fra en reguleringsplan. Oppsummeringen skal være så "
            "kort som mulig, men fortsatt tydelig og informativ. Unngå unødvendige "
            "detaljer. Dersom versjon 1 inneholder tekst, men ikke versjon 2, betyr "
            "det at teksten har blitt fjernet. Hvis det er omvendt har teksten blitt "
            "lagt til. Ikke referer til versjon 1 eller versjon 2 i oppsummeringen. "
            "Beskriv kun hva som er fjernet, lagt til eller endret."
        )
    ),
    HumanMessage(
        content=f"""\
Bruk eksemplene som hjelp til å velge riktig ord.

{formatted_examples}
               
Oppsummer forskjellene mellom følgende to versjoner:

Versjon 1:
{version_1}

Versjon 2:
{version_2}

Oppsummering:
"""
    )
]

    try: 
        response = llm.invoke(msg)

        results.append({
            'id': item['id'],
            'ref_summary': ref_summary,
            'model_summary': response.content
        })

    except Exception as e:
        print(f"Error: {e}")
        continue

100%|██████████| 35/35 [05:06<00:00,  8.76s/it]


In [19]:
with open(base_dir / f"results/{config['OPENAI_DEPLOYMENT_NAME']}/{config['OPENAI_DEPLOYMENT_NAME']}_TESTSET.jsonl", "w") as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')